
I'll also include instructions for remote monitoring over Wi-Fi.

---

## **Step-by-Step Instructions**

### **Prerequisites**

- **Raspberry Pi Username:** `hpfed`
- **Virtual Environment Name:** `HOMEPHOTOFED`
- **Python Script Name:** `fed3_ttl_listener.py`
- **Script Location:** We will place it in `/home/hpfed/scripts/`

### **Assumptions**

- You have administrative (sudo) access to the Raspberry Pi.
- Your Python script is tested and working correctly.
- All required Python packages are installed in the `HOMEPHOTOFED` virtual environment.

---

### **Step 1: Place Your Script in an Appropriate Location**

1. **Create a directory for your scripts (if it doesn't exist):**

   ```bash
   mkdir -p /home/hpfed/scripts
   ```

2. **Copy your Python script to this directory:**

   You can transfer the script using `scp`, `rsync`, or by editing it directly on the Raspberry Pi.

   **Example using `nano`:**

   ```bash
   nano /home/hpfed/scripts/fed3_ttl_listener.py
   ```

   Paste your script into the editor, save (`Ctrl+O`), and exit (`Ctrl+X`).

3. **Ensure the script is executable:**

   ```bash
   chmod +x /home/hpfed/scripts/fed3_ttl_listener.py
   ```

---

### **Step 2: Verify Your Virtual Environment**

1. **Activate your virtual environment:**

   ```bash
   source /home/hpfed/HOMEPHOTOFED/bin/activate
   ```

2. **Ensure all required packages are installed:**

   Install any necessary packages (if not already installed):

   ```bash
   pip install RPi.GPIO pyserial
   ```

3. **Test your script within the virtual environment:**

   ```bash
   python /home/hpfed/scripts/fed3_ttl_listener.py
   ```

   - Ensure it runs without errors.
   - Press `Ctrl+C` to stop the script.

4. **Deactivate the virtual environment:**

   ```bash
   deactivate
   ```

---

### **Step 3: Create a `systemd` Service File**

1. **Create a new service file:**

   ```bash
   sudo nano /etc/systemd/system/fed3_ttl_listener.service
   ```

2. **Add the following content to the service file:**

   ```ini
   [Unit]
   Description=FED3 TTL Listener Service
   After=network.target

   [Service]
   Type=simple
   ExecStart=/bin/bash -c 'source /home/hpfed/HOMEPHOTOFED/bin/activate && exec python /home/hpfed/scripts/fed3_ttl_listener.py'
   WorkingDirectory=/home/hpfed/scripts
   StandardOutput=syslog
   StandardError=syslog
   SyslogIdentifier=fed3_ttl_listener
   User=hpfed
   Group=hpfed
   Restart=always
   RestartSec=10

   [Install]
   WantedBy=multi-user.target
   ```

   **Explanation:**

   - **ExecStart:** Activates the virtual environment and runs the script.
   - **User/Group:** Runs the service as your user `hpfed`.
   - **Restart:** Automatically restarts the service if it crashes.
   - **RestartSec:** Waits 10 seconds before restarting.

3. **Save and exit the editor** (`Ctrl+O`, `Enter`, `Ctrl+X`).

---

### **Step 4: Configure Logging (Optional but Recommended)**

1. **Create a log configuration file for `rsyslog`:**

   ```bash
   sudo nano /etc/rsyslog.d/fed3_ttl_listener.conf
   ```

2. **Add the following content:**

   ```
   if $programname == 'fed3_ttl_listener' then /var/log/fed3_ttl_listener.log
   & stop
   ```

   **Explanation:**

   - Redirects logs from your service to `/var/log/fed3_ttl_listener.log`.

3. **Save and exit the editor** (`Ctrl+O`, `Enter`, `Ctrl+X`).

4. **Create the log file and set permissions:**

   ```bash
   sudo touch /var/log/fed3_ttl_listener.log
   sudo chown hpfed:hpfed /var/log/fed3_ttl_listener.log
   ```

5. **Restart `rsyslog` to apply changes:**

   ```bash
   sudo systemctl restart rsyslog
   ```

---

### **Step 5: Reload `systemd` and Enable the Service**

1. **Reload `systemd` to recognize the new service file:**

   ```bash
   sudo systemctl daemon-reload
   ```

2. **Enable the service to start on boot:**

   ```bash
   sudo systemctl enable fed3_ttl_listener.service
   ```

3. **Start the service now:**

   ```bash
   sudo systemctl start fed3_ttl_listener.service
   ```

4. **Check the status of the service:**

   ```bash
   sudo systemctl status fed3_ttl_listener.service
   ```

   - Ensure it shows `Active: active (running)`.
   - If there are errors, they will be displayed here.

---

### **Step 6: Test the Service**

1. **Check the service logs:**

   ```bash
   tail -f /var/log/fed3_ttl_listener.log
   ```

   - Verify that your script is running and producing the expected output.
   - Press `Ctrl+C` to exit the log monitoring.

2. **Check for errors in the `journalctl`:**

   ```bash
   sudo journalctl -u fed3_ttl_listener.service
   ```

---

### **Step 7: Reboot and Verify**

1. **Reboot the Raspberry Pi:**

   ```bash
   sudo reboot
   ```

2. **After reboot, ensure the service is running:**

   ```bash
   sudo systemctl status fed3_ttl_listener.service
   ```

3. **Check the logs again to ensure the script is running correctly:**

   ```bash
   tail -f /var/log/fed3_ttl_listener.log
   ```

---

### **Step 8: Remote Monitoring Over Wi-Fi**

#### **1. Enable SSH on the Raspberry Pi**

```bash
sudo raspi-config
```

- Navigate to **Interface Options** > **SSH** > **Enable**.
- Exit the configuration tool.

#### **2. Connect Both Devices to the Same Network**

- Ensure your Raspberry Pi and your PC are connected to the same Wi-Fi network.

#### **3. Find Your Raspberry Pi's IP Address**

On the Raspberry Pi, run:

```bash
hostname -I
```

- Note the IP address (e.g., `192.168.1.100`).

#### **4. Connect to the Raspberry Pi from Your PC**

- **From a Linux or macOS PC:**

  ```bash
  ssh hpfed@<raspberry_pi_ip_address>
  ```

- **From a Windows PC:**

  - Use an SSH client like **PuTTY**.
  - - download PuTTY [Here](https://apps.microsoft.com/detail/xpfnzksklbp7rj?hl=en-us&gl=NO)
  - Enter the IP address and connect as user `hpfed`.

#### **5. Monitor the Service and Logs Remotely**

- **Check the service status:**

  ```bash
  sudo systemctl status fed3_ttl_listener.service
  ```

- **Monitor the logs:**

  ```bash
  tail -f /var/log/fed3_ttl_listener.log
  ```

---

### **Additional Considerations**

#### **Permissions and User Configuration**

- **Ensure `hpfed` is in the `gpio` and `dialout` groups:**

  ```bash
  sudo usermod -aG gpio,dialout hpfed
  ```

  - Log out and log back in for the changes to take effect.

#### **Verify Paths and Names**

- **Virtual Environment Path:**

  - Confirm that the virtual environment activation script is at `/home/hpfed/HOMEPHOTOFED/bin/activate`.

- **Script Path:**

  - Ensure your script is at `/home/hpfed/scripts/fed3_ttl_listener.py`.

#### **Testing the Script Manually**

- Activate the virtual environment:

  ```bash
  source /home/hpfed/HOMEPHOTOFED/bin/activate
  ```

- Run the script:

  ```bash
  python /home/hpfed/scripts/fed3_ttl_listener.py
  ```

- If the script runs without errors, proceed to test it via the service.

#### **Service Troubleshooting**

- **View detailed logs:**

  ```bash
  sudo journalctl -u fed3_ttl_listener.service
  ```

- **Check for syntax errors in the service file.**

#### **Security Considerations**

- **SSH Security:**

  - Consider changing the default password for user `hpfed` if you haven't already.

- **Firewall Settings:**

  - If you have a firewall enabled, ensure it allows SSH connections.

---

### **Final Notes**

- **Hardware Connections:**

  - Ensure all GPIO connections are secure and correctly wired.
  - Double-check pin numbers and voltage levels to prevent damage.

- **Power Supply:**

  - Use a reliable power source for the Raspberry Pi to prevent unexpected shutdowns.

- **Backup:**

  - It's good practice to back up your configuration files and scripts.

---

## **Summary**

By following these steps, you have:

- Configured your Raspberry Pi to automatically run your Python script at startup within the `HOMEPHOTOFED` virtual environment.
- Set up a `systemd` service to manage your script.
- Configured logging to monitor your script's output.
- Enabled remote access to your Raspberry Pi for monitoring and management.

---

**If you have any questions or need further assistance, please don't hesitate to ask. I'm here to help!**

Certainly! I'd be happy to provide a detailed explanation on how to use an SSH client like **PuTTY** to connect to your Raspberry Pi from a Windows PC. This will allow you to remotely access your Raspberry Pi's terminal over a network connection, enabling you to monitor logs, manage services, and perform other tasks without needing a direct physical connection.

---

## **Step-by-Step Guide: Using PuTTY to Connect to Your Raspberry Pi**

### **Prerequisites**

- **Raspberry Pi** connected to your local network (via Wi-Fi or Ethernet).
- **Windows PC** connected to the same local network.
- **SSH Enabled** on your Raspberry Pi.
- **PuTTY** installed on your Windows PC.

---

### **1. Enable SSH on Your Raspberry Pi**

**Note**: If you have already enabled SSH on your Raspberry Pi, you can skip this step.

1. **Open the Terminal on Your Raspberry Pi**:

   - If you have a monitor and keyboard connected to your Raspberry Pi, open the Terminal application.
   - If not, you can enable SSH by adding an empty file named `ssh` (without any extension) to the `/boot` partition of your SD card using another computer.

2. **Run `raspi-config`**:

   ```bash
   sudo raspi-config
   ```

3. **Navigate to Interface Options**:

   - Use the arrow keys to select **Interface Options** and press **Enter**.

4. **Enable SSH**:

   - Select **SSH** and press **Enter**.
   - Choose **Yes** to enable SSH.
   - Press **Enter** to confirm.

5. **Finish and Reboot**:

   - Navigate to **Finish** and reboot your Raspberry Pi if prompted.

---

### **2. Find Your Raspberry Pi's IP Address**

You'll need the IP address of your Raspberry Pi to connect via SSH.

1. **Run the Following Command on Your Raspberry Pi**:

   ```bash
   hostname -I
   ```

   - This command will display the IP address(es) of your Raspberry Pi.
   - **Example Output**: `192.168.1.100`

2. **Note the IP Address**:

   - Write down the IP address; you'll need it to connect from PuTTY.

---

### **3. Download and Install PuTTY on Your Windows PC**

1. **Download PuTTY**:

   - Visit the official PuTTY download page: [https://www.putty.org/](https://www.putty.org/)
   - Click on **"You can download PuTTY here"** to go to the download page.
   - Under **"Package files"**, download the **64-bit x86** installer (e.g., `putty-64bit-<version>-installer.msi`).

     - If your PC is 32-bit, download the **32-bit x86** installer instead.

2. **Install PuTTY**:

   - Run the downloaded installer.
   - Follow the on-screen instructions to install PuTTY.
   - The default installation options are suitable for most users.

---

### **4. Use PuTTY to Connect to Your Raspberry Pi**

1. **Launch PuTTY**:

   - Open the **Start** menu and search for **PuTTY**.
   - Click on **PuTTY** to launch the application.

2. **Configure the Connection**:

   - **Session Category**:

     - In the left pane, ensure **Session** is selected.

   - **Host Name (or IP address)**:

     - Enter the IP address of your Raspberry Pi (e.g., `192.168.1.100`).

   - **Port**:

     - Ensure the port is set to `22` (default SSH port).

   - **Connection Type**:

     - Ensure **SSH** is selected.

   - **Saved Sessions (Optional)**:

     - You can save this session for future use by entering a name (e.g., `RaspberryPi`) and clicking **Save**.

3. **Configure SSH Settings (Optional)**:

   - For most users, default settings are sufficient.
   - If you need to adjust settings, you can explore categories in the left pane, such as **Connection** > **SSH**.

4. **Start the SSH Session**:

   - Click on **Open** at the bottom of the PuTTY window.

5. **Security Alert**:

   - On first connection, you may see a **PuTTY Security Alert** stating that the server's host key is not cached.

   - **This is normal on the first connection**.

   - Click **Yes** to trust the connection and proceed.

6. **Login Prompt**:

   - A terminal window will open with a black background.

   - **Login as**:

     - Enter your Raspberry Pi username: `hpfed`

     - Press **Enter**.

   - **Password**:

     - Enter your Raspberry Pi password.

     - **Note**: For security, you won't see any characters or placeholders as you type the password. Just type it carefully and press **Enter**.

7. **Successful Connection**:

   - If the credentials are correct, you will be logged in, and the command prompt will display:

     ```bash
     hpfed@raspberrypi:~$
     ```

---

### **5. Use the SSH Session to Manage Your Raspberry Pi**

Now that you're connected, you can perform various tasks remotely.

#### **Check the Status of Your Service**

```bash
sudo systemctl status fed3_ttl_listener.service
```

- This will display the status of your Python script service.
- Use the arrow keys to scroll; press **Q** to exit the status view.

#### **View the Logs**

```bash
tail -f /var/log/fed3_ttl_listener.log
```

- This command displays the last few lines of your log file and updates in real-time.
- Press **Ctrl+C** to exit the log view.

#### **Stop or Restart the Service**

- **Stop the Service**:

  ```bash
  sudo systemctl stop fed3_ttl_listener.service
  ```

- **Restart the Service**:

  ```bash
  sudo systemctl restart fed3_ttl_listener.service
  ```

#### **Run Other Commands**

- Navigate directories, edit files, and manage your system just as you would from the local terminal.

---

### **6. Disconnecting the SSH Session**

- To end the SSH session, simply type:

  ```bash
  exit
  ```

- Or, you can close the PuTTY window directly.

---

### **7. Transferring Files Between Your PC and Raspberry Pi (Optional)**

If you need to transfer files, you can use **WinSCP**, which provides a graphical interface for file transfers over SSH.

#### **Install WinSCP**

1. **Download WinSCP**:

   - Visit [https://winscp.net/](https://winscp.net/eng/download.php)

   - Download the installer package for Windows.

2. **Install WinSCP**:

   - Run the installer and follow the prompts.

#### **Use WinSCP to Transfer Files**

1. **Launch WinSCP**.

2. **Login Dialog**:

   - **File protocol**: Select **SCP**.

   - **Host name**: Enter your Raspberry Pi's IP address.

   - **Port number**: Ensure it is `22`.

   - **User name**: Enter `hpfed`.

   - **Password**: Enter your Raspberry Pi password.

   - **Save session (Optional)**:

     - You can save the session for future use.

   - Click **Login**.

3. **Security Alert**:

   - You may receive a warning about the host key.

   - Click **Yes** to proceed.

4. **Transfer Files**:

   - The interface shows your local files on the left and the Raspberry Pi's files on the right.

   - You can navigate directories and drag-and-drop files between your PC and Raspberry Pi.

---

### **8. Troubleshooting Connection Issues**

#### **Connection Refused or Timeout**

- **Verify IP Address**:

  - Ensure you're using the correct IP address.

- **Raspberry Pi Powered On**:

  - Make sure the Raspberry Pi is turned on and connected to the network.

- **SSH Enabled**:

  - Double-check that SSH is enabled on the Raspberry Pi.

- **Network Connectivity**:

  - Ensure both devices are on the same network and subnet.

#### **Authentication Failed**

- **Username and Password**:

  - Verify that you're using the correct username (`hpfed`) and password.

- **Caps Lock**:

  - Ensure that Caps Lock is not interfering with your password entry.

#### **Firewall or Security Software**

- **Windows Firewall**:

  - Check if Windows Firewall is blocking PuTTY.

  - Allow PuTTY through the firewall if necessary.

- **Antivirus Software**:

  - Some security software may interfere with SSH connections.

---

### **9. Additional Tips**

#### **Use SSH Keys for Authentication (Advanced)**

- For increased security, you can set up SSH key-based authentication.

- This eliminates the need to enter a password each time.

- **Guide**: [Setting up SSH keys on the Raspberry Pi](https://www.raspberrypi.org/documentation/remote-access/ssh/passwordless.md)

#### **Change the Default Password**

- If you haven't already, change the default password for user `hpfed` to enhance security:

  ```bash
  passwd
  ```

- Follow the prompts to set a new password.

#### **Assign a Static IP Address (Optional)**

- Assigning a static IP to your Raspberry Pi can make it easier to connect in the future.

- You can set this up via your router's DHCP settings or configure it on the Raspberry Pi.

#### **Use Hostnames Instead of IP Addresses**

- You can use the Raspberry Pi's hostname instead of the IP address if your network supports it.

- Default hostname is `raspberrypi`, so you can try connecting to `raspberrypi.local`.

- **Example in PuTTY**:

  - **Host Name**: `raspberrypi.local`

---

### **10. Summary**

By following these steps, you can:

- Remotely access your Raspberry Pi's terminal from a Windows PC using PuTTY.
- Manage services, view logs, and perform administrative tasks without needing a direct connection.
- Transfer files between your PC and Raspberry Pi using WinSCP.

---

### **Need Further Assistance?**

If you have any questions or encounter any issues while setting up or using PuTTY to connect to your Raspberry Pi, please feel free to ask. I'm here to help you troubleshoot and ensure you can manage your Raspberry Pi effectively.

Certainly! Let’s address your concerns step-by-step to ensure you have full control over your Raspberry Pi, whether it's operating headlessly (without peripherals) or connected to a screen, keyboard, and mouse.

---

## **Overview**

- **Auto-Running Script**: Your Python script (`fed3_ttl_listener.py`) is set up to run automatically at startup using a `systemd` service.
- **Accessing Raspberry Pi**: Whether your Pi is headless or connected to peripherals, you can manage and control it effectively.
- **Stopping/Disabling the Auto-Run Script**: You have full control to stop or disable the script from running automatically whenever needed.

---

## **1. Understanding `systemd` Services**

`systemd` is a system and service manager for Linux operating systems, including Raspberry Pi OS. It manages how services start, stop, and behave on your system.

- **Service File**: Defines how `systemd` manages your script.
- **Enabled Service**: Starts automatically at boot.
- **Disabled Service**: Does not start automatically; needs manual start.

---

## **2. Managing the Auto-Running Python Script**

### **A. Stopping the Service Temporarily**

If you want to **stop** the script from running **temporarily** (until the next reboot or until you start it again):

1. **Open Terminal or SSH into Your Pi**:
   
   - **Headless**: Use SSH (see Section 3).
   - **With Peripherals**: Open the Terminal application.

2. **Stop the Service**:

   ```bash
   sudo systemctl stop fed3_ttl_listener.service
   ```

   - **Explanation**: This command stops the service immediately. The script will cease running until you start the service again or reboot the Pi.

3. **Verify the Service is Stopped**:

   ```bash
   sudo systemctl status fed3_ttl_listener.service
   ```

   - **Look for**: `Active: inactive (dead)`

### **B. Disabling the Service Permanently**

If you want to **prevent** the script from running **automatically** at startup:

1. **Disable the Service**:

   ```bash
   sudo systemctl disable fed3_ttl_listener.service
   ```

   - **Explanation**: This command disables the service, so it won’t start automatically on boot.

2. **Stop the Service (if it's currently running)**:

   ```bash
   sudo systemctl stop fed3_ttl_listener.service
   ```

3. **Verify the Service is Disabled**:

   ```bash
   sudo systemctl status fed3_ttl_listener.service
   ```

   - **Look for**: `Loaded: loaded (/etc/systemd/system/fed3_ttl_listener.service; disabled; ...)`

### **C. Enabling the Service Again**

If you previously disabled the service and want to **re-enable** it:

1. **Enable the Service**:

   ```bash
   sudo systemctl enable fed3_ttl_listener.service
   ```

2. **Start the Service Immediately**:

   ```bash
   sudo systemctl start fed3_ttl_listener.service
   ```

3. **Verify the Service is Running**:

   ```bash
   sudo systemctl status fed3_ttl_listener.service
   ```

   - **Look for**: `Active: active (running)`

---

## **3. Accessing Your Raspberry Pi Remotely**

### **A. Headless Access via SSH**

**SSH (Secure Shell)** allows you to remotely access the Raspberry Pi’s terminal over your local network.

#### **1. Enable SSH (If Not Already Enabled)**

If SSH wasn’t enabled during initial setup:

1. **Access the SD Card on Another Computer**:
   
   - Insert the Raspberry Pi’s SD card into another computer.
   - Open the `boot` partition.

2. **Create an Empty File Named `ssh`**:
   
   - In the `boot` directory, create a file named `ssh` **without any extension**.
   - This enables SSH on boot.

3. **Safely Eject the SD Card and Reinsert It into the Pi**.

#### **2. Find the Raspberry Pi’s IP Address**

1. **From the Pi** (if you have temporary peripheral access):

   ```bash
   hostname -I
   ```

2. **From Your Router**:

   - Log into your router’s admin interface.
   - Check the list of connected devices to find the Pi’s IP.

3. **Using Network Scanning Tools** (from your PC):

   - Tools like **Advanced IP Scanner** (Windows) or `nmap` (Linux/macOS).

#### **3. Connect Using PuTTY**

1. **Download and Install PuTTY** (if not already installed):

   - [Download PuTTY](https://www.putty.org/)

2. **Launch PuTTY**.

3. **Configure the Connection**:

   - **Host Name (or IP address)**: Enter the Pi’s IP address (e.g., `192.168.1.100`).
   - **Port**: `22` (default for SSH).
   - **Connection Type**: Ensure **SSH** is selected.

4. **Save the Session (Optional)**:

   - Enter a name under **Saved Sessions** (e.g., `RaspberryPi`).
   - Click **Save** for quick access later.

5. **Start the Session**:

   - Click **Open**.

6. **Security Alert**:

   - On first connection, PuTTY will display a security alert about the server’s host key.
   - Click **Yes** to trust and continue.

7. **Login Prompt**:

   - **Username**: `hpfed`
   - **Password**: Enter your Pi’s password.

8. **Successful Connection**:

   - You should see a terminal prompt like:
     
     ```bash
     hpfed@raspberrypi:~$
     ```

### **B. Access with Peripherals (Screen, Keyboard, Mouse)**

If you occasionally connect peripherals:

1. **Connect Screen, Keyboard, and Mouse to the Pi**.

2. **Power On the Pi**.

3. **Log In Directly**:

   - Use the `hpfed` username and your password to log in.

4. **Manage Services Locally**:

   - You can run `systemctl` commands directly from the terminal.

---

## **4. Accessing the Rest of the Raspberry Pi**

**Having your script run automatically does not restrict access to the Raspberry Pi's other functionalities.** Here's how you can interact with the Pi alongside the script:

### **A. Using SSH**

- **Multiple Sessions**: You can have multiple SSH sessions open simultaneously.
- **Running Commands**: Execute other commands, manage files, or monitor system status without interfering with the script.

### **B. Using Peripherals**

- **GUI Access**: If you connect a screen, you can use the Raspberry Pi’s desktop environment alongside the script.
- **Terminal Access**: Open multiple terminal windows or tabs to manage different tasks.

### **C. Monitoring and Managing the Script**

- **Check Status**:

  ```bash
  sudo systemctl status fed3_ttl_listener.service
  ```

- **View Logs**:

  ```bash
  tail -f /var/log/fed3_ttl_listener.log
  ```

- **Stop/Start/Restart the Service**:

  ```bash
  sudo systemctl stop fed3_ttl_listener.service
  sudo systemctl start fed3_ttl_listener.service
  sudo systemctl restart fed3_ttl_listener.service
  ```

---

## **5. Additional Tips**

### **A. Ensuring Script Runs Smoothly on Boot**

1. **Auto-Start Without Peripherals**:

   - By setting up the `systemd` service correctly, your script will run in the background regardless of peripherals being connected.

2. **No GUI Interference**:

   - Since the script runs as a service, it operates independently of the graphical desktop environment.

### **B. Managing Resource Usage**

- **Monitor System Performance**:

  ```bash
  top
  ```

- **Check for Errors in Logs**:

  - Regularly review `/var/log/fed3_ttl_listener.log` for any issues.

### **C. Securing Your Raspberry Pi**

- **Change Default Password** (if not already done):

  ```bash
  passwd
  ```

- **Keep the System Updated**:

  ```bash
  sudo apt update
  sudo apt upgrade -y
  ```

- **Limit SSH Access**:

  - Consider using SSH keys instead of passwords for enhanced security.

### **D. Remote File Management**

- **Using SCP with PuTTY’s PSCP**:

  - [Download PSCP](https://www.chiark.greenend.org.uk/~sgtatham/putty/latest.html)
  - Transfer files between your PC and Pi via command line.

- **Using WinSCP for GUI-Based File Transfer**:

  - [Download WinSCP](https://winscp.net/)
  - Connect using the same SSH credentials to drag and drop files.

---

## **6. Summary**

- **Auto-Running Script**: Your Python script is managed by a `systemd` service, allowing it to run automatically at boot.
- **Stopping/Disabling**: You can stop or disable the script’s service at any time using `systemctl` commands.
- **Access Flexibility**: Whether headless or connected to peripherals, you can access and manage your Raspberry Pi seamlessly.
- **Remote Monitoring**: Use SSH clients like PuTTY to remotely monitor logs, manage services, and perform other tasks over Wi-Fi.
- **Safety and Security**: Always ensure your Raspberry Pi is secure, especially when enabling remote access.

---

## **Final Thoughts**

By setting up your Raspberry Pi with a `systemd` service, you've enabled it to function autonomously, handling data from FEDs and communicating with the TDT system seamlessly. Simultaneously, you retain full control and access to the Pi's other functionalities, whether you're connected directly or managing it remotely.

If you follow the steps outlined above, you’ll ensure that your setup is robust, flexible, and secure. However, if you encounter any challenges or have further questions, feel free to reach out!

---

**Happy Pi-ing! 🥧**

# <font color = "Yellow">Cell below how to enable access to Juypter lab installed on RBPI from your remote desktop computer connection <font />


Certainly! Running Jupyter Lab on a headless Raspberry Pi (without a connected monitor, keyboard, or mouse) and accessing it remotely from your PC is entirely feasible. It sounds like you're encountering an issue where Jupyter Lab isn't displaying as expected on your PC. Let's walk through the complete setup to ensure you can access Jupyter Lab seamlessly from your PC.

---

## **Overview**

1. **Install Jupyter Lab in Your Virtual Environment**
2. **Configure Jupyter Lab for Remote Access**
3. **Set Up SSH Tunneling for Secure Access**
4. **Access Jupyter Lab from Your PC**
5. **Optional: Running Jupyter Lab as a `systemd` Service**
6. **Troubleshooting Common Issues**

---

## **1. Install Jupyter Lab in Your Virtual Environment**

Ensure that Jupyter Lab is installed within your `HOMEPHOTOFED` virtual environment. This ensures that all dependencies are managed correctly.

### **Steps:**

1. **Activate Your Virtual Environment:**

   ```bash
   source /home/hpfed/HOMEPHOTOFED/bin/activate
   ```

2. **Install Jupyter Lab:**

   ```bash
   pip install jupyterlab
   ```

3. **Verify Installation:**

   ```bash
   jupyter lab --version
   ```

   - This should display the installed version of Jupyter Lab.

4. **Deactivate the Virtual Environment:**

   ```bash
   deactivate
   ```

---

## **2. Configure Jupyter Lab for Remote Access**

By default, Jupyter Lab is configured to only accept connections from `localhost` (i.e., the Raspberry Pi itself). To access it remotely, you need to adjust its configuration.

### **Steps:**

1. **Activate Your Virtual Environment:**

   ```bash
   source /home/hpfed/HOMEPHOTOFED/bin/activate
   ```

2. **Generate Jupyter Configuration File:**

   ```bash
   jupyter lab --generate-config
   ```

   - This creates a `jupyter_lab_config.py` file in `/home/hpfed/.jupyter/`.

3. **Set a Password for Jupyter Lab:**

   It's essential to secure Jupyter Lab with a password to prevent unauthorized access.

   ```bash
   jupyter lab password
   ```

   - Follow the prompts to enter and verify your desired password.

4. **Edit the Configuration File:**

   Open the `jupyter_lab_config.py` file in a text editor.

   ```bash
   nano /home/hpfed/.jupyter/jupyter_lab_config.py
   ```

   **Modify/Add the Following Lines:**

   ```python
   # Allow all IP addresses to access Jupyter Lab
   c.ServerApp.ip = '0.0.0.0'

   # Specify a port (default is 8888)
   c.ServerApp.port = 8888

   # Disable launching a browser on the server
   c.ServerApp.open_browser = False

   # Allow root access if needed (optional)
   # c.ServerApp.allow_root = True

   # Optional: Set a custom base URL
   # c.ServerApp.base_url = '/jupyter/'
   ```

   **Save and Exit:**
   
   - Press `Ctrl + O`, then `Enter` to save.
   - Press `Ctrl + X` to exit.

5. **(Optional) Enable HTTPS for Secure Connections:**

   If you're accessing Jupyter Lab over an unsecured network, it's recommended to enable HTTPS.

   - **Generate SSL Certificates:**

     ```bash
     mkdir /home/hpfed/.jupyter/ssl
     openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout /home/hpfed/.jupyter/ssl/jupyter.key -out /home/hpfed/.jupyter/ssl/jupyter.crt
     ```

     - Follow the prompts to enter certificate details.

   - **Update Configuration File:**

     Add the following lines to `jupyter_lab_config.py`:

     ```python
     c.ServerApp.certfile = '/home/hpfed/.jupyter/ssl/jupyter.crt'
     c.ServerApp.keyfile = '/home/hpfed/.jupyter/ssl/jupyter.key'
     ```

6. **Deactivate the Virtual Environment:**

   ```bash
   deactivate
   ```

---

## **3. Set Up SSH Tunneling for Secure Access**

SSH tunneling allows you to securely access Jupyter Lab without exposing it directly to the network. This is a recommended approach for security.

### **Steps:**

1. **Ensure SSH is Enabled on Your Raspberry Pi:**

   You've likely already done this, but to confirm:

   ```bash
   sudo systemctl status ssh
   ```

   - If it's not active, enable it:

     ```bash
     sudo systemctl enable ssh
     sudo systemctl start ssh
     ```

2. **From Your PC, Set Up an SSH Tunnel Using PuTTY:**

   Since you're using PuTTY, here's how to configure SSH tunneling.

   #### **A. Open PuTTY and Enter Basic Connection Details:**

   - **Host Name (or IP address):** Enter your Raspberry Pi's IP address (e.g., `192.168.1.100`).
   - **Port:** `22`
   - **Connection Type:** Ensure **SSH** is selected.

   #### **B. Configure SSH Tunneling:**

   1. **Navigate to SSH Tunnels Settings:**

      - In the left pane, expand **SSH** > **Tunnels**.

   2. **Set Up the Tunnel:**

      - **Source port:** `8888` (or another local port if `8888` is in use).
      - **Destination:** `localhost:8888`
      - **Type:** Ensure **Local** is selected.

      - **Add the Tunnel:**

        Click **Add**. You should see a new entry like `L8888 localhost:8888` in the **Forwarded ports** list.

   3. **Save the Session (Optional):**

      - Go back to **Session** in the left pane.
      - Under **Saved Sessions**, enter a name (e.g., `RaspberryPiJupyter`).
      - Click **Save** for future use.

   4. **Start the SSH Session:**

      Click **Open** to initiate the connection.

   5. **Login Prompt:**

      - **Username:** `hpfed`
      - **Password:** Enter your Raspberry Pi password.

   6. **Verify the Tunnel:**

      - Once logged in, any connection to `localhost:8888` on your PC will be securely tunneled to `localhost:8888` on your Raspberry Pi.

3. **Access Jupyter Lab from Your PC's Web Browser:**

   - Open your preferred web browser on your PC.
   - Navigate to: [http://localhost:8888](http://localhost:8888)
   
   **Note:** If you set a different source port in PuTTY (e.g., `9999`), replace `8888` with that port number (e.g., [http://localhost:9999](http://localhost:9999)).

   - **Enter the Password:**

     Use the password you set during the Jupyter Lab configuration.

   - **Success:**

     You should now see the Jupyter Lab interface, fully operational.

---

## **4. Access Jupyter Lab Directly Over the Network (Alternative Method)**

**Warning:** This method exposes Jupyter Lab directly to your network. It's less secure than SSH tunneling and should only be used in trusted networks or with proper security measures (like HTTPS and strong passwords).

### **Steps:**

1. **Ensure Your Firewall Allows Incoming Connections on Port 8888:**

   ```bash
   sudo ufw allow 8888
   ```

2. **Start Jupyter Lab in the Virtual Environment:**

   ```bash
   source /home/hpfed/HOMEPHOTOFED/bin/activate
   jupyter lab
   ```

   - **Note:** Since you have configured Jupyter Lab to run on `0.0.0.0`, it will accept connections from any network interface.

3. **Access Jupyter Lab from Your PC's Web Browser:**

   - Navigate to: `http://<raspberry_pi_ip_address>:8888`
   - Enter the password you set.

4. **(Optional) Use HTTPS for Secure Access:**

   - If you've set up SSL certificates, use `https://` instead of `http://`.
   - Example: `https://192.168.1.100:8888`

---

## **5. Optional: Running Jupyter Lab as a `systemd` Service**

To ensure Jupyter Lab starts automatically at boot and runs in the background, you can set it up as a `systemd` service.

### **Steps:**

1. **Create a Service File:**

   ```bash
   sudo nano /etc/systemd/system/jupyter_lab.service
   ```

2. **Add the Following Content:**

   ```ini
   [Unit]
   Description=Jupyter Lab
   After=network.target

   [Service]
   Type=simple
   User=hpfed
   Group=hpfed
   WorkingDirectory=/home/hpfed/scripts
   ExecStart=/bin/bash -c 'source /home/hpfed/HOMEPHOTOFED/bin/activate && exec jupyter lab'
   Restart=always
   RestartSec=10

   [Install]
   WantedBy=multi-user.target
   ```

   **Explanation:**

   - **ExecStart:** Activates the virtual environment and starts Jupyter Lab.
   - **User/Group:** Runs the service as `hpfed`.
   - **Restart:** Automatically restarts Jupyter Lab if it crashes.
   - **WorkingDirectory:** Directory from which Jupyter Lab runs (adjust if necessary).

3. **Save and Exit:**

   - Press `Ctrl + O`, then `Enter` to save.
   - Press `Ctrl + X` to exit.

4. **Reload `systemd` to Recognize the New Service:**

   ```bash
   sudo systemctl daemon-reload
   ```

5. **Enable the Service to Start on Boot:**

   ```bash
   sudo systemctl enable jupyter_lab.service
   ```

6. **Start the Service Now:**

   ```bash
   sudo systemctl start jupyter_lab.service
   ```

7. **Check the Service Status:**

   ```bash
   sudo systemctl status jupyter_lab.service
   ```

   - Ensure it shows `Active: active (running)`.
   - If there are errors, they will be displayed here.

8. **View Jupyter Lab Logs:**

   Since the service redirects output to syslog, check the logs:

   ```bash
   sudo journalctl -u jupyter_lab.service -f
   ```

   - Press `Ctrl + C` to stop viewing logs.

---

## **6. Troubleshooting Common Issues**

### **Issue 1: Unable to Connect via SSH Tunnel**

**Possible Causes:**

- **Incorrect IP Address:** Ensure you're using the correct IP address of your Raspberry Pi.
- **Firewall Blocking Port:** Verify that port `22` (SSH) is open and not blocked by any firewall.
- **SSH Service Not Running:** Confirm that SSH is active.

**Solutions:**

1. **Verify SSH Service:**

   ```bash
   sudo systemctl status ssh
   ```

   - If inactive, start it:

     ```bash
     sudo systemctl start ssh
     ```

2. **Check Raspberry Pi's IP Address:**

   ```bash
   hostname -I
   ```

3. **Ensure SSH Tunnel is Set Up Correctly:**

   - Revisit the PuTTY SSH tunnel configuration to ensure it's forwarding `localhost:8888` correctly.

### **Issue 2: Jupyter Lab Not Accessible**

**Possible Causes:**

- **Jupyter Lab Not Running:** Ensure the service is active.
- **Incorrect Configuration:** Verify the `jupyter_lab_config.py` settings.
- **Port Conflicts:** Ensure no other service is using port `8888`.

**Solutions:**

1. **Check Jupyter Lab Service Status:**

   ```bash
   sudo systemctl status jupyter_lab.service
   ```

2. **Review Logs for Errors:**

   ```bash
   sudo journalctl -u jupyter_lab.service -f
   ```

3. **Change Jupyter Lab Port (If Needed):**

   - Edit `jupyter_lab_config.py` and set a different port.

4. **Ensure No Port Conflicts:**

   ```bash
   sudo lsof -i -P -n | grep LISTEN
   ```

   - Look for any service using port `8888`.

### **Issue 3: Authentication Errors**

**Possible Causes:**

- **Incorrect Password:** Ensure you're using the correct Jupyter Lab password.
- **SSH Credentials:** Verify SSH username and password.

**Solutions:**

1. **Reset Jupyter Lab Password:**

   ```bash
   source /home/hpfed/HOMEPHOTOFED/bin/activate
   jupyter lab password
   deactivate
   ```

2. **Change Raspberry Pi Password:**

   ```bash
   passwd
   ```

   - Follow the prompts to set a new password.

---

## **Summary**

By following these steps, you can:

- **Run Jupyter Lab Automatically:** Configure it to start on boot within your virtual environment.
- **Access Jupyter Lab Remotely:** Use SSH tunneling with PuTTY to securely access Jupyter Lab from your PC's web browser.
- **Manage and Monitor Remotely:** Whether headless or connected to peripherals, you retain full control over your Raspberry Pi and its services.
- **Ensure Security:** By setting strong passwords and using SSH tunneling, you maintain a secure environment for your data and operations.

---

## **Additional Resources**

- **Jupyter Documentation:** [https://jupyter.org/documentation](https://jupyter.org/documentation)
- **Raspberry Pi SSH Guide:** [https://www.raspberrypi.org/documentation/remote-access/ssh/](https://www.raspberrypi.org/documentation/remote-access/ssh/)
- **PuTTY Documentation:** [https://www.putty.org/](https://www.putty.org/)
- **Raspberry Pi OS Documentation:** [https://www.raspberrypi.org/documentation/](https://www.raspberrypi.org/documentation/)

---

Feel free to reach out if you encounter any issues or have further questions. I'm here to help you ensure a smooth and efficient setup!